# Structure

Imputation with kNN
- 1. no column creation
    - 1.1. raw data
        - LR, RF, XGboost, LightGBM, Catboost, Neural Network
    - 1.2. ADASYN imputation
        - LR, RF, XGboost, LightGBM, Catboost, Neural Network
- 2. yes column creation
    - 2.1. raw data
        - LR, RF, XGboost, LightGBM, Catboost, Neural Network
    - 2.2. ADASYN imputation
        - LR, RF, XGboost, LightGBM, Catboost, Neural Network

## Load the data and the packages

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
from tensorflow import keras
from tensorflow.keras import layers
from imblearn.over_sampling import ADASYN
from fancyimpute import KNN

(CVXPY) Oct 25 04:08:48 PM: Encountered unexpected exception importing solver CVXOPT:
ImportError("dlopen(/Users/minwukim/anaconda3/lib/python3.11/site-packages/cvxopt/base.cpython-311-darwin.so, 0x0002): Library not loaded: @rpath/liblapack.3.dylib\n  Referenced from: <E25E40AB-7857-39B9-8DE7-28B7B0E4806B> /Users/minwukim/anaconda3/lib/python3.11/site-packages/cvxopt/base.cpython-311-darwin.so\n  Reason: tried: '/Users/minwukim/anaconda3/lib/python3.11/site-packages/cvxopt/../../../liblapack.3.dylib' (no such file), '/Users/minwukim/anaconda3/lib/python3.11/site-packages/cvxopt/../../../liblapack.3.dylib' (no such file), '/Users/minwukim/anaconda3/bin/../lib/liblapack.3.dylib' (no such file), '/Users/minwukim/anaconda3/bin/../lib/liblapack.3.dylib' (no such file), '/usr/local/lib/liblapack.3.dylib' (no such file), '/usr/lib/liblapack.3.dylib' (no such file, not in dyld cache)")
(CVXPY) Oct 25 04:08:48 PM: Encountered unexpected exception importing solver GLPK:
ImportError("dlopen(/Use

In [2]:
df = pd.read_csv('../../database/2016-2022_semantic_imputation.csv')

In [3]:
binary = ['unequal_voting','classified_board_system','poison_pill','operating_margin_below_3y_average']
non_ratio_variables = [
    "capex",
    "net_capex",
    "short_term_wc",
    "long_term_wc",
    "modified_wc",
    "ebitda",
    "ebit",
    "net_income",
    "net_debt",
    "ev",
    "repurchase",
    "board_size",
    "net_repurchase",
    "total_compensation_to_executives",
    "total_compensation_to_board_members",
    "dividend_to_common",
    "dividend_to_preferred"
]

df['ev_ebitda'] = np.where((df['ev'] != 0) & (df['ebitda'] != 0), df['ev'] / df['ebitda'], np.nan)
df['ev_ebit'] = np.where((df['ev'] != 0) & (df['ebit'] != 0), df['ev'] / df['ebit'], np.nan)

ratio_variables = [
    "ebitda_margin",
    "operating_margin",
    "sales_to_total_assets",
    "roe",
    "normalized_roe",
    "operating_roe",
    "operating_roic",
    "eps_adjusted_diluted",
    "ev_to_sales",
    "tobin_q_ratio",
    "pb_ratio",
    "pe_ratio",
    "fcf_to_equity",
    "sales_growth_rate",
    "dividend_per_share",
    "dividend_payout_ratio",
    "asset_to_equity",
    "cash_conversion_cycle",
    "ev_ebitda",
    "ev_ebit",
]

technical_variables = [
    "free_float_percentage",
    "rsi_14d",
    "rsi_30d",
    "volatility_30d",
    "volatility_90d",
    "volatility_180d",
    "volume_30d_average_to_outstanding",
    "insider_shares_percentage",
    "institution_ownership_percentage",
    "ceo_tenure",
    "total_return_5y",
    "total_return_4y",
    "total_return_3y",
    "total_return_2y",
    "total_return_1y",
    "total_return_6m",
    "total_return_3m",
    "employee_growth_rate",
    "fcf_yield"
]

supportive = ["bic_level_2","bic_level_3","market_cap"]
factors = binary + non_ratio_variables + ratio_variables + technical_variables

df["bic_level_2"] = df["bic_level_2"].astype('category')
df["bic_level_3"] = df["bic_level_3"].astype('category')

# factors.append("targeted")

## 1. No column creation

### 1.1. raw data

In [4]:
knn_imputer = KNN(k=5)
df_imputed = pd.DataFrame(knn_imputer.fit_transform(df[factors]), columns=factors, index=df[factors].index)

Imputing row 1/18213 with 5 missing, elapsed time: 75.085
Imputing row 101/18213 with 2 missing, elapsed time: 75.157
Imputing row 201/18213 with 3 missing, elapsed time: 75.196
Imputing row 301/18213 with 4 missing, elapsed time: 75.228
Imputing row 401/18213 with 1 missing, elapsed time: 75.263
Imputing row 501/18213 with 0 missing, elapsed time: 75.309
Imputing row 601/18213 with 1 missing, elapsed time: 75.364
Imputing row 701/18213 with 5 missing, elapsed time: 75.398
Imputing row 801/18213 with 3 missing, elapsed time: 75.432
Imputing row 901/18213 with 1 missing, elapsed time: 75.467
Imputing row 1001/18213 with 3 missing, elapsed time: 75.510
Imputing row 1101/18213 with 2 missing, elapsed time: 75.553
Imputing row 1201/18213 with 3 missing, elapsed time: 75.593
Imputing row 1301/18213 with 6 missing, elapsed time: 75.648
Imputing row 1401/18213 with 0 missing, elapsed time: 75.680
Imputing row 1501/18213 with 14 missing, elapsed time: 75.711
Imputing row 1601/18213 with 5 miss

Imputing row 13801/18213 with 1 missing, elapsed time: 80.878
Imputing row 13901/18213 with 2 missing, elapsed time: 80.917
Imputing row 14001/18213 with 4 missing, elapsed time: 80.951
Imputing row 14101/18213 with 5 missing, elapsed time: 80.996
Imputing row 14201/18213 with 1 missing, elapsed time: 81.044
Imputing row 14301/18213 with 1 missing, elapsed time: 81.081
Imputing row 14401/18213 with 0 missing, elapsed time: 81.119
Imputing row 14501/18213 with 7 missing, elapsed time: 81.148
Imputing row 14601/18213 with 0 missing, elapsed time: 81.188
Imputing row 14701/18213 with 9 missing, elapsed time: 81.224
Imputing row 14801/18213 with 1 missing, elapsed time: 81.254
Imputing row 14901/18213 with 6 missing, elapsed time: 81.293
Imputing row 15001/18213 with 5 missing, elapsed time: 81.331
Imputing row 15101/18213 with 9 missing, elapsed time: 81.368
Imputing row 15201/18213 with 5 missing, elapsed time: 81.421
Imputing row 15301/18213 with 2 missing, elapsed time: 81.457
Imputing

In [5]:
df = pd.concat([df_imputed, df[['year', 'targeted',"market_cap","bic_level_2","bic_level_3"]]], axis=1)

In [6]:
train_data = df[df['year'].isin([2016, 2017, 2018, 2019, 2020])]
test_data = df[df['year'] == 2021]

X_train = train_data[factors]
y_train = train_data['targeted']

X_test = test_data[factors]
y_test = test_data['targeted']

# Scaling data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

def create_nn():
    model = keras.Sequential([
        layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
        layers.Dropout(0.5),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy')
    return model

# Models (without neural network for now)
models = {
    "LR": LogisticRegression(max_iter=10000),
    "RF": RandomForestClassifier(n_estimators=100),
    "XGB": xgb.XGBClassifier(),
    "LGBM": lgb.LGBMClassifier(),
    "CatBoost": cb.CatBoostClassifier(verbose=0, iterations=100)
}

# Setup for Cross Validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=14)

# Storage for AUC scores
train_aucs = {}
test_aucs = {}

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred_train = cross_val_predict(model, X_train, y_train, cv=cv, method='predict_proba')[:, 1]
    y_pred_test = model.predict_proba(X_test)[:, 1]
    
    train_aucs[model_name] = roc_auc_score(y_train, y_pred_train)
    test_aucs[model_name] = roc_auc_score(y_test, y_pred_test)

# Neural Network
nn_model = create_nn()
nn_model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=0)
y_pred_train_nn = nn_model.predict(X_train_scaled)
y_pred_test_nn = nn_model.predict(X_test_scaled)
train_aucs["NN"] = roc_auc_score(y_train, y_pred_train_nn)
test_aucs["NN"] = roc_auc_score(y_test, y_pred_test_nn)

# Compile Results
result1 = pd.DataFrame({
    'Model': list(train_aucs.keys()),
    'Train AUC': list(train_aucs.values()),
    'Test AUC': list(test_aucs.values())
})
print(result1)


82/82 [==============================] - 0s 308us/step
      Model  Train AUC  Test AUC
0        LR   0.495903  0.476470
1        RF   0.622747  0.593968
2       XGB   0.647005  0.610015
3      LGBM   0.660908  0.632027
4  CatBoost   0.652073  0.660517
5        NN   0.922107  0.610481


### 1.2. oversampling with ADASYN

In [7]:
# Split data
train_data = df[df['year'].isin([2016, 2017, 2018, 2019, 2020])]
test_data = df[df['year'] == 2021]

X_train = train_data[factors]
y_train = train_data['targeted']

X_test = test_data[factors]
y_test = test_data['targeted']

# Apply ADASYN oversampling
adasyn = ADASYN(random_state=42)
X_train, y_train = adasyn.fit_resample(X_train, y_train)


In [8]:
# Scaling data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

def create_nn():
    model = keras.Sequential([
        layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
        layers.Dropout(0.5),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy')
    return model

# Models (without neural network for now)
models = {
    "LR": LogisticRegression(max_iter=10000),
    "RF": RandomForestClassifier(n_estimators=100),
    "XGB": xgb.XGBClassifier(),
    "LGBM": lgb.LGBMClassifier(),
    "CatBoost": cb.CatBoostClassifier(verbose=0, iterations=100)
}

# Setup for Cross Validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=14)

# Storage for AUC scores
train_aucs = {}
test_aucs = {}

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred_train = cross_val_predict(model, X_train, y_train, cv=cv, method='predict_proba')[:, 1]
    y_pred_test = model.predict_proba(X_test)[:, 1]
    
    train_aucs[model_name] = roc_auc_score(y_train, y_pred_train)
    test_aucs[model_name] = roc_auc_score(y_test, y_pred_test)

# Neural Network
nn_model = create_nn()
nn_model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=0)
y_pred_train_nn = nn_model.predict(X_train_scaled)
y_pred_test_nn = nn_model.predict(X_test_scaled)
train_aucs["NN"] = roc_auc_score(y_train, y_pred_train_nn)
test_aucs["NN"] = roc_auc_score(y_test, y_pred_test_nn)

# Compile Results
result2 = pd.DataFrame({
    'Model': list(train_aucs.keys()),
    'Train AUC': list(train_aucs.values()),
    'Test AUC': list(test_aucs.values())
})
print(result2)

82/82 [==============================] - 0s 305us/step
      Model  Train AUC  Test AUC
0        LR   0.571568  0.502407
1        RF   0.999380  0.616202
2       XGB   0.997711  0.634982
3      LGBM   0.995835  0.617289
4  CatBoost   0.995536  0.586793
5        NN   0.998469  0.571224


## 2.1 with column creation

In [9]:
df = pd.read_csv('../../database/2016-2022_semantic_imputation.csv')
df['ev_ebitda'] = np.where((df['ev'] != 0) & (df['ebitda'] != 0), df['ev'] / df['ebitda'], np.nan)
df['ev_ebit'] = np.where((df['ev'] != 0) & (df['ebit'] != 0), df['ev'] / df['ebit'], np.nan)

In [10]:
for col in non_ratio_variables:
    
    # 1. _percentile
    percentile_col = col + '_percentile'
    df[percentile_col] = df.groupby('year')[col].transform(lambda x: x.rank(pct=True) * 100)
    df[percentile_col].fillna(50, inplace=True)
    
    # 2. _10bins_percentile
    df['market_cap_bins'] = df.groupby('year')['market_cap'].transform(lambda x: pd.cut(x, bins=10))
    percentile_10bins_col = col + '_10bins_percentile'
    df[percentile_10bins_col] = df.groupby(['year', 'market_cap_bins'])[col].transform(lambda x: x.rank(pct=True) * 100)
    df[percentile_10bins_col].fillna(50, inplace=True)
    df.drop('market_cap_bins', axis=1, inplace=True)

    # 3. _10bins_normalized
    df['market_cap_bins'] = df.groupby('year')['market_cap'].transform(lambda x: pd.qcut(x, 10, labels=False, duplicates='drop'))
    normalized_col = col + '_10bins_normalized'
    df[normalized_col] = df.groupby(['year', 'market_cap_bins'])[col].transform(lambda x: (x - x.mean()) / x.std())
    df[normalized_col].fillna(0, inplace=True)
    df.drop('market_cap_bins', axis=1, inplace=True)
    
    # 4. _div_market_cap
    div_market_cap_col = col + '_div_market_cap'
    df[div_market_cap_col] = df[col] / df['market_cap']
    
    # 5. _div_log_market_cap
    df['log_market_cap'] = np.log(df['market_cap'])
    div_log_market_cap_col = col + '_div_log_market_cap'
    df[div_log_market_cap_col] = df[col] / df['log_market_cap']
    

In [11]:
def compute_percentile(group):
    if len(group) < 10:
        return pd.Series([None] * len(group), index=group.index, dtype=float)
    return group.rank(pct=True) * 100

def normalize(group):
    if len(group) < 10:
        return pd.Series([None] * len(group), index=group.index, dtype=float)
    return (group - group.mean()) / group.std()

for col in ratio_variables:
    percentile_col = col + '_industry_peers_percentile'
    df[percentile_col] = df.groupby(['year', 'bic_level_3'])[col].transform(compute_percentile)
    mask = df[percentile_col].isna()
    df.loc[mask, percentile_col] = df[mask].groupby(['year', 'bic_level_2'])[col].transform(compute_percentile)
    df[percentile_col].fillna(50, inplace=True)
    df[percentile_col] = df[percentile_col].astype(float)
    normalized_col = col + '_industry_peers_normalized'
    df[normalized_col] = df.groupby(['year', 'bic_level_3'])[col].transform(normalize)
    mask = df[normalized_col].isna()
    df.loc[mask, normalized_col] = df[mask].groupby(['year', 'bic_level_2'])[col].transform(normalize)
    df[normalized_col].fillna(0, inplace=True)
    df[normalized_col] = df[normalized_col].astype(float)


In [12]:
factors = []
for col in non_ratio_variables:
    factors.extend([
        col,
        f'{col}_percentile',
        f'{col}_10bins_percentile',
        f'{col}_10bins_normalized',
        f'{col}_div_market_cap',
        f'{col}_div_log_market_cap'
    ])

for col in ratio_variables:
    factors.extend([
        col,
        f'{col}_industry_peers_percentile',
        f'{col}_industry_peers_normalized'
    ])

factors = factors + binary + technical_variables


In [13]:
knn_imputer = KNN(k=5)
df_imputed = pd.DataFrame(knn_imputer.fit_transform(df[factors]), columns=factors, index=df[factors].index)
df = pd.concat([df_imputed, df[['year', 'targeted',"market_cap","bic_level_2","bic_level_3"]]], axis=1)

Imputing row 1/18213 with 7 missing, elapsed time: 155.718
Imputing row 101/18213 with 6 missing, elapsed time: 155.837
Imputing row 201/18213 with 7 missing, elapsed time: 155.908
Imputing row 301/18213 with 8 missing, elapsed time: 155.970
Imputing row 401/18213 with 1 missing, elapsed time: 156.032
Imputing row 501/18213 with 0 missing, elapsed time: 156.116
Imputing row 601/18213 with 1 missing, elapsed time: 156.188
Imputing row 701/18213 with 11 missing, elapsed time: 156.253
Imputing row 801/18213 with 7 missing, elapsed time: 156.317
Imputing row 901/18213 with 1 missing, elapsed time: 156.380
Imputing row 1001/18213 with 7 missing, elapsed time: 156.463
Imputing row 1101/18213 with 6 missing, elapsed time: 156.545
Imputing row 1201/18213 with 7 missing, elapsed time: 156.619
Imputing row 1301/18213 with 16 missing, elapsed time: 156.693
Imputing row 1401/18213 with 0 missing, elapsed time: 156.751
Imputing row 1501/18213 with 30 missing, elapsed time: 156.809
Imputing row 1601

Imputing row 13301/18213 with 0 missing, elapsed time: 165.463
Imputing row 13401/18213 with 1 missing, elapsed time: 165.515
Imputing row 13501/18213 with 0 missing, elapsed time: 165.584
Imputing row 13601/18213 with 23 missing, elapsed time: 165.662
Imputing row 13701/18213 with 23 missing, elapsed time: 165.722
Imputing row 13801/18213 with 1 missing, elapsed time: 165.770
Imputing row 13901/18213 with 2 missing, elapsed time: 165.831
Imputing row 14001/18213 with 12 missing, elapsed time: 165.885
Imputing row 14101/18213 with 11 missing, elapsed time: 165.977
Imputing row 14201/18213 with 1 missing, elapsed time: 166.041
Imputing row 14301/18213 with 1 missing, elapsed time: 166.102
Imputing row 14401/18213 with 0 missing, elapsed time: 166.162
Imputing row 14501/18213 with 17 missing, elapsed time: 166.207
Imputing row 14601/18213 with 0 missing, elapsed time: 166.275
Imputing row 14701/18213 with 15 missing, elapsed time: 166.318
Imputing row 14801/18213 with 1 missing, elapsed 

### 2.1. raw data

In [14]:
train_data = df[df['year'].isin([2016, 2017, 2018, 2019, 2020])]
test_data = df[df['year'] == 2021]

X_train = train_data[factors]
y_train = train_data['targeted']

X_test = test_data[factors]
y_test = test_data['targeted']

# Scaling data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

def create_nn():
    model = keras.Sequential([
        layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
        layers.Dropout(0.5),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy')
    return model

# Models (without neural network for now)
models = {
    "LR": LogisticRegression(max_iter=10000),
    "RF": RandomForestClassifier(n_estimators=100),
    "XGB": xgb.XGBClassifier(),
    "LGBM": lgb.LGBMClassifier(),
    "CatBoost": cb.CatBoostClassifier(verbose=0, iterations=100)
}

# Setup for Cross Validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=14)

# Storage for AUC scores
train_aucs = {}
test_aucs = {}

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred_train = cross_val_predict(model, X_train, y_train, cv=cv, method='predict_proba')[:, 1]
    y_pred_test = model.predict_proba(X_test)[:, 1]
    
    train_aucs[model_name] = roc_auc_score(y_train, y_pred_train)
    test_aucs[model_name] = roc_auc_score(y_test, y_pred_test)

# Neural Network
nn_model = create_nn()
nn_model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=0)
y_pred_train_nn = nn_model.predict(X_train_scaled)
y_pred_test_nn = nn_model.predict(X_test_scaled)
train_aucs["NN"] = roc_auc_score(y_train, y_pred_train_nn)
test_aucs["NN"] = roc_auc_score(y_test, y_pred_test_nn)

# Compile Results
result3 = pd.DataFrame({
    'Model': list(train_aucs.keys()),
    'Train AUC': list(train_aucs.values()),
    'Test AUC': list(test_aucs.values())
})
print(result3)


/Users/minwukim/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/minwukim/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

82/82 [==============================] - 0s 332us/step
      Model  Train AUC  Test AUC
0        LR   0.524704  0.502557
1        RF   0.595857  0.603905
2       XGB   0.652837  0.659762
3      LGBM   0.664124  0.642298
4  CatBoost   0.648389  0.647255
5        NN   0.943861  0.616496


### 2.2. oversampling the ADASYN

In [15]:
train_data = df[df['year'].isin([2016, 2017, 2018, 2019, 2020])]
test_data = df[df['year'] == 2021]

X_train = train_data[factors]
y_train = train_data['targeted']

X_test = test_data[factors]
y_test = test_data['targeted']

# Apply ADASYN oversampling
adasyn = ADASYN(random_state=42)
X_train, y_train = adasyn.fit_resample(X_train, y_train)

# Scaling data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

def create_nn():
    model = keras.Sequential([
        layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
        layers.Dropout(0.5),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy')
    return model

# Models (without neural network for now)
models = {
    "LR": LogisticRegression(max_iter=10000),
    "RF": RandomForestClassifier(n_estimators=100),
    "XGB": xgb.XGBClassifier(),
    "LGBM": lgb.LGBMClassifier(),
    "CatBoost": cb.CatBoostClassifier(verbose=0, iterations=100)
}

# Setup for Cross Validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=14)

# Storage for AUC scores
train_aucs = {}
test_aucs = {}

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred_train = cross_val_predict(model, X_train, y_train, cv=cv, method='predict_proba')[:, 1]
    y_pred_test = model.predict_proba(X_test)[:, 1]
    
    train_aucs[model_name] = roc_auc_score(y_train, y_pred_train)
    test_aucs[model_name] = roc_auc_score(y_test, y_pred_test)

# Neural Network
nn_model = create_nn()
nn_model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=0)
y_pred_train_nn = nn_model.predict(X_train_scaled)
y_pred_test_nn = nn_model.predict(X_test_scaled)
train_aucs["NN"] = roc_auc_score(y_train, y_pred_train_nn)
test_aucs["NN"] = roc_auc_score(y_test, y_pred_test_nn)

# Compile Results
result4 = pd.DataFrame({
    'Model': list(train_aucs.keys()),
    'Train AUC': list(train_aucs.values()),
    'Test AUC': list(test_aucs.values())
})
print(result4)

82/82 [==============================] - 0s 338us/step
      Model  Train AUC  Test AUC
0        LR   0.593854  0.540768
1        RF   0.999734  0.636969
2       XGB   0.998261  0.637804
3      LGBM   0.996751  0.643373
4  CatBoost   0.995105  0.606466
5        NN   0.999999  0.624232
